In [21]:
import numpy as np
import random
import math

In [30]:
def rho(x,p):
    ans = (1-p)*math.log2(1+(1-2*p)*x) + p*math.log2(1-(1-2*p)*x)
    return ans

# Variable Length BZ Algorithm
# takes input array ar, tolerated prob Pe, and crossover prob p
def VLBZ(ar,num,Pe,p):
    # current step
    k = 0
    n = len(ar)
    q = np.ones(n)/n
    Z = 1/(1-q)
    while np.max(Z)<=math.log2(1/Pe):
        k = k + 1
        sum = 0
        j = -1
        while sum<0.5:
            j = j + 1
            sum = sum + q[j]
        t1 = 2*(1-sum+q[j]-0.5)
        t2 = 2*(sum-0.5)
        j_star = j - sub
        t = 2*(sum-ar[j])-1 if j_star==j-1 else 2*(sum)-1
        sub = 1 if random.random()<pi1 else 0
        pi1 = (rho(t2,p)-rho(-t,p2))/(rho(t1,p)-rho(-t,p1)+rho(t2,p)-rho(-t,p2))
        Y = 1 if ar[j_star]<num else 0
        N = 1 if random.random()<p else 0
        Y = Y ^ N
        f1 = np.ones(n)
        for i in range(n):
            f1[i] = (2*(1-p)*q[i])/(1+(1-2*Y)(1-2*p)*t)
            cond[i] = ((j_star<=i) == (Y))
        q[cond] = f1[cond]
        q[~cond] = f1[~cond]
        Z = q/(1-q)
    
    l = np.argmax(q)
    return l

In [31]:
print(VLBZ([1,3,8,12],7,0.2,0.3))

UnboundLocalError: local variable 'sub' referenced before assignment